# 参数管理

* 首先看一下具有单隐藏层的多层感知机。

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1678],
        [-0.0788]], grad_fn=<AddmmBackward0>)

**1. 参数访问**

* 从已有模型中访问参数。当通过`Sequential`类定义模型时，可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。如下所示，可以检查第二个全连接层的参数。
* 首先，这个全连接层包含两个参数，分别是该层的权重和偏置。两者都存储为单精度浮点数（float32）。注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0557, -0.2587,  0.1445, -0.3315,  0.2848,  0.2984,  0.0983,  0.0193]])), ('bias', tensor([-0.0761]))])


**2. 目标参数**

* 每个参数都表示为参数类的一个实例。要对参数执行任何操作，首先需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，提取后返回的是一个参数类实例，并进一步访问该参数的值。

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0761], requires_grad=True)
tensor([-0.0761])


* 参数是复合的对象，包含值、梯度和额外信息。这就是我们需要显式参数值的原因。
除了值之外，还可以访问每个参数的梯度。在上面这个网络中，由于还没有调用反向传播，所以参数的梯度处于初始状态。

In [4]:
net[2].weight.grad == None

True

**3. 一次性访问所有参数**

* 当需要对所有参数执行操作时，逐个访问它们可能会很麻烦。当处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，因为需要递归整个树来提取每个子块的参数。

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


* 另一种方式

In [6]:
net.state_dict()['2.bias'].data

tensor([-0.0761])

**4. 从嵌套块收集参数**

* 首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套4个block1()
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2177],
        [-0.2177]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


# 参数初始化

* 默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入和输出维度计算出的。PyTorch的`nn.init`模块提供了多种预置初始化方法。

**1. 内置初始化**

* 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，且将偏置参数设置为0。

In [9]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)  
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0105, -0.0032, -0.0034, -0.0208]), tensor(0.))

* 还可以将所有参数初始化为给定的常数，比如初始化为1。

In [10]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

* **对某些块应用不同的初始化方法**。例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，然后将第三个神经网络层初始化为常量值42。

In [11]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1521, -0.3630, -0.1432, -0.5545])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


**2. 自定义初始化**

* 有时，深度学习框架没有提供我们需要的初始化方法。在下面的例子中，使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -9.4662, -7.4129,  0.0000],
        [ 9.1692,  5.6209, -8.5521, -6.2768]], grad_fn=<SliceBackward0>)

* 注意，始终可以直接设置参数。

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -8.4662, -6.4129,  1.0000])

**3. 参数绑定**

* 有时希望在多个层间共享参数：可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [15]:
# 需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
# 即修改了一层的数据也会修改另一层
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
